#### LeetCode - SQL - #176

Refer [176. Second Highest Salary](https://leetcode.com/problems/second-highest-salary/description/)

Write a solution to find the second highest salary from the `Employee` table. If there is no second highest salary, return `null`.

In [0]:
data = [[1, 100], [2, 200], [2, 200], [3, 300]]

columns = ['id', 'salary']
schema = {'id':'int64', 'salary':'int64'}

In [0]:
import pandas

salary_pandas_dataframe = pandas.DataFrame(data=data, columns=columns).astype(schema)

In [0]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName("LeetCode SQL").getOrCreate()

# Spark DataFrame using Pandas DataFrame
salary_dataframe = spark_context.createDataFrame(salary_pandas_dataframe)
salary_dataframe.printSchema()
salary_dataframe.show(5)

root
 |-- id: long (nullable = true)
 |-- salary: long (nullable = true)

+---+------+
| id|salary|
+---+------+
|  1|   100|
|  2|   200|
|  2|   200|
|  3|   300|
+---+------+



#### Using normal SQL functions

In [0]:
from pyspark.sql.functions import max

# 1
max_salary = salary_dataframe.select(max(salary_dataframe.salary)).collect()[0][0]
print(max_salary)

salary_dataframe \
    .where(salary_dataframe.salary != max_salary) \
    .select(max(salary_dataframe.salary).alias("SecondHighestSalary")) \
    .show()

# 2
salary_dataframe \
    .select(salary_dataframe.salary).distinct() \
    .sort(salary_dataframe.salary.desc()) \
    .offset(1) \
    .limit(1) \
    .show()

+------+
|salary|
+------+
|   200|
+------+



#### Using Window Fuctions

In [0]:
from pyspark.sql.functions import dense_rank, max
from pyspark.sql.window import Window

windowSpec = Window.orderBy(salary_dataframe.salary.desc())
salary_dataframe = salary_dataframe.withColumn("salary_rank", dense_rank().over(windowSpec))
salary_dataframe.show(5)

salary_dataframe \
    .filter(salary_dataframe.salary_rank==2) \
    .select(max(salary_dataframe.salary).alias("SecondHighestSalary")) \
    .show()

+---+------+-----------+
| id|salary|salary_rank|
+---+------+-----------+
|  3|   300|          1|
|  2|   200|          2|
|  2|   200|          2|
|  1|   100|          3|
+---+------+-----------+

+-------------------+
|SecondHighestSalary|
+-------------------+
|                200|
+-------------------+

